单独的大模型只能给出文本信息，没有后续操作，langchain可以通过定义“工具调用”的过程，响应LLMs的输出，扩展其能力。下面是为LLMs绑定工具的过程

In [1]:
from langchain_ollama import ChatOllama
# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "qwen2.5:7b"
llm  = ChatOllama(model=model_name)

query = "What is 3 * 10? Also, what is 11 + 49?"
response=llm.invoke(query)
response

AIMessage(content="Sure, let's solve these calculations:\n\nFor the first calculation:\n\\[ 3 \\times 10 = 30 \\]\n\nAnd for the second calculation:\n\\[ 11 + 49 = 60 \\]", additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-10-11T08:31:05.228678629Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 696602694, 'load_duration': 29402530, 'prompt_eval_count': 50, 'prompt_eval_duration': 47974000, 'eval_count': 48, 'eval_duration': 473583000}, id='run-bfdba99e-57ce-4b6a-9378-3fa9b461f187-0', usage_metadata={'input_tokens': 50, 'output_tokens': 48, 'total_tokens': 98})

In [2]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools) # 为LLMs绑定工具
query = "What is 3 * 12?"
response=llm_with_tools.invoke(query)
response

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-10-11T08:31:07.211280254Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 499541548, 'load_duration': 30210556, 'prompt_eval_count': 226, 'prompt_eval_duration': 68868000, 'eval_count': 26, 'eval_duration': 251383000}, id='run-195295d7-21a9-461e-abb6-54e53cd85187-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'aa697f87-4b36-4cd0-bfd8-dab8ea80400d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 226, 'output_tokens': 26, 'total_tokens': 252})

为LLMs绑定工具后，不像前面一样直接给出了答案(content="")，而且多了信息输出"'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}]"，这里LLMs通过query“找到要调用的函数及其输入参数”，对于多个问答也被解析出来。这里就很神奇LLMs知道自己要执行的函数，实际上它是根据函数名去匹配的。以下即使提问多个实例，也可以被正确解析

In [3]:
query = "What is 3 * 12? Also, what is 11 + 49?"

response=llm_with_tools.invoke(query).tool_calls
response

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': '099346fe-83b7-437d-a8b5-6d38efee71c8',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': '2b55b9be-8c09-475c-ad22-95f74502d765',
  'type': 'tool_call'}]

以上调用LLMs只返回了“待调用的函数及参数”，并没有真正执行计算，也就是没有得到最终结果。为了得到最终结果，需要调用外部函数计算，然后将计算结果再次返回给LLMs，进行最后的总结

In [7]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"
messages = [HumanMessage(query)]
print(messages)

ai_msg = llm_with_tools.invoke(messages)
print(ai_msg) # AIMessage类
print(ai_msg.tool_calls)

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={})]
content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-10-11T08:32:42.815706058Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}, {'function': {'name': 'add', 'arguments': {'a': 11, 'b': 49}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 777538299, 'load_duration': 29805739, 'prompt_eval_count': 238, 'prompt_eval_duration': 50489000, 'eval_count': 55, 'eval_duration': 546107000} id='run-addeaa94-e9bc-44f8-9c19-02bca7258bdf-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'a27c01b0-bb91-4cbf-b943-f5089ed68c3f', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '1d7b5df7-dc62-4aea-9378-c3c9414392ff', 'type': 'tool_call'}] usage_metadata={'input_tokens': 238, 'output_tokens': 55, 'total_tokens': 293}
[{'nam

LLMs解析出输入query要执行的函数，以下遍历调用拿到结果，并结果信息收集和之前的输入合并

In [5]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    print(tool_call,tool_msg) # AIMessage类 
    messages.append(tool_msg)
print(messages)

{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '18caab1e-4f8b-4cee-9fd9-a5e1ac753f9d', 'type': 'tool_call'} content='36' name='multiply' tool_call_id='18caab1e-4f8b-4cee-9fd9-a5e1ac753f9d'
{'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'ba28072a-38fa-406a-93ae-bd88e15229e6', 'type': 'tool_call'} content='60' name='add' tool_call_id='ba28072a-38fa-406a-93ae-bd88e15229e6'
[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}), ToolMessage(content='36', name='multiply', tool_call_id='18caab1e-4f8b-4cee-9fd9-a5e1ac753f9d'), ToolMessage(content='60', name='add', tool_call_id='ba28072a-38fa-406a-93ae-bd88e15229e6')]


合并后的信息再次输入LLMs，综合输出结果

In [6]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of 3 multiplied by 12 is 36. And the sum of 11 and 49 is 60.', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-10-11T08:31:31.056073291Z', 'message': {'role': 'assistant', 'content': 'The result of 3 multiplied by 12 is 36. And the sum of 11 and 49 is 60.'}, 'done_reason': 'stop', 'done': True, 'total_duration': 676565885, 'load_duration': 35597521, 'prompt_eval_count': 257, 'prompt_eval_duration': 41573000, 'eval_count': 32, 'eval_duration': 311393000}, id='run-5e64ef68-b2b5-4cdc-a355-cff9af3595c7-0', usage_metadata={'input_tokens': 257, 'output_tokens': 32, 'total_tokens': 289})

即使是组合运算，LLMs也能解析出来要调用的函数，langchain真的神奇

In [43]:
query = "What is 11*2 + 49?"
response=llm_with_tools.invoke(query).tool_calls
print(response)

[{'name': 'multiply', 'args': {'a': 11, 'b': 2}, 'id': 'a459426a-77f9-4a30-bb7b-8f0f9c29ed15', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 22, 'b': 49}, 'id': '791a6cf3-01cd-4cc8-8d57-67498475aa3f', 'type': 'tool_call'}]
